<a href="https://colab.research.google.com/github/ozgurakgun/notebooks/blob/main/5_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For context, see https://twitter.com/wtgowers/status/1581667648592826369

Let's make a copy of it here as well.

---

> A set of 5 numbers has:
> - mode of 24
> - median of 21
> - mean of 20
>
> What are the 5 numbers?

---

There is some excitement about the wording of this question on Twitter, which is fair. There isn't a unique solution and from the wording it might sound like there would be. Let's ignore that for a moment.

(I am also slightly disturbed by the use of "set" here, even though we are not finding a mathematical set, where all elements would be distinct. Oh well.)

---

Let's do some thinking here.

Solutions would look like this: a,b,c,d,e (a list of 5 numbers)

To avoid equivalent solutions, let's post that this list of numbers will be in non-decreasing order.

Median is 21, so c=21.

Mode is 24, so it must be the most frequent. 24 can only occur to the right of c, since our list is ordered. And we need at least two 24s for it to be more frequent than 21. And we need can have at most two 24s since there are only 2 numbers to the right of 21. So d=e=24.

Mean is 20, hence sum is 100. `c+d+e=69`. `a+b=31`. `b<21`. So the following are all solutions:

- a=11, b=20
- a=12, b=19
- a=13, b=18
- a=14, b=17
- a=15, b=16

The problem has 5 solutions.

---

We can also use Constraint Programming to solve this problem. Let's model and solve using Conjure.

We first load the Conjure extension for Jupyter.

In [1]:
!source <(curl -s https://raw.githubusercontent.com/conjure-cp/conjure-notebook/v0.0.10/scripts/install-colab.sh)
%reload_ext conjure

Installing Conjure version v2.5.1 and Conjure Notebook version v0.0.10...
Downloading...
Conjure: The Automated Constraint Modelling Tool
Release version 2.5.1
Repository version a9cbc2e (2023-11-07 23:44:00 +0000)


<IPython.core.display.Javascript object>

Conjure extension is loaded - run `%conjure_help`

We will want to enumerate all solutions, for that we use the Conjure settings command and make the following two changes:
- turn off solution printing
- type "all" in the number of solutions box. this box takes either a number or the string all.

We need to give a lower and an upper bound for the numbers, Essence deals with finite domain problems only. This is not explicitly given in the problem description, so we need to come up with something. It wouldn't hurt to use large numbers (for correctness). The only problem could be efficiency, in some cases, but this is an easy problem.

In [2]:
lb, ub = 0, 100

Now we write a model!

In [3]:
%%conjure

given lb, ub : int
letting number be domain int(lb..ub)
find x : matrix indexed by [int(1..5)] of number


```json
{"x": {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0}}
```

| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.001 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 0.237 |
| SolverFailures | 0 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SolverTimeOut | 0 |
| SolverNodes | 6 |


In [4]:
print(x)

{'1': 0, '2': 0, '3': 0, '4': 0, '5': 0}


In [5]:
def conjure_display_solution():
    print(x)
    print(f"Sum is {sum(x.values())}")

In [6]:
%%conjure+

$ mean is 20
such that sum(x) = 5 * 20


{'1': 0, '2': 0, '3': 0, '4': 0, '5': 100}
Sum is 100


| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.001 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 0.155 |
| SolverFailures | 0 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SolverTimeOut | 0 |
| SolverNodes | 5 |


In [7]:
%%conjure+

$ median is 21
such that x[3] = 21


{'1': 0, '2': 0, '3': 21, '4': 0, '5': 79}
Sum is 100


| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.001 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 0.18 |
| SolverFailures | 0 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SolverTimeOut | 0 |
| SolverNodes | 4 |


In [8]:
%%conjure+

$ mode is 24
such that [ freq(x, 24) > freq(x, i) | i : number, i != 24 ]


{'1': 0, '2': 24, '3': 21, '4': 24, '5': 31}
Sum is 100


| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.010 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 1.643 |
| SolverFailures | 46 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SolverTimeOut | 0 |
| SolverNodes | 51 |


In [9]:
%%conjure+

$ symmetry breaking
such that [ x[i-1] <= x[i] | i : int(2..5) ]


{'1': 11, '2': 20, '3': 21, '4': 24, '5': 24}
Sum is 100


| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.000 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 0.671 |
| SolverFailures | 0 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SolverTimeOut | 0 |
| SolverNodes | 2 |


In [10]:
%%conjure+ --number-of-solutions=all

such that true


## Solution 1

{'1': 11, '2': 20, '3': 21, '4': 24, '5': 24}
Sum is 100


## Solution 2

{'1': 12, '2': 19, '3': 21, '4': 24, '5': 24}
Sum is 100


## Solution 3

...

## Solution 4

{'1': 14, '2': 17, '3': 21, '4': 24, '5': 24}
Sum is 100


## Solution 5

{'1': 15, '2': 16, '3': 21, '4': 24, '5': 24}
Sum is 100


| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.004 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 0.562 |
| SolverFailures | 5 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SolverTimeOut | 0 |
| SolverNodes | 5 |


In [11]:
# just some hacky Python to print things out
# importantly, notice that conjure_solutions is available in Python

def printSol(sol):
  return ", ".join([ str(y) for x, y in sol.items() ])

print("\n".join(sorted([printSol(sol['x']) for sol in conjure_solutions])))

11, 20, 21, 24, 24
12, 19, 21, 24, 24
13, 18, 21, 24, 24
14, 17, 21, 24, 24
15, 16, 21, 24, 24
